# Parallel GNN Model for Oral Bioavailability Dataset 

1. This notebook focuses on building the parallel GNN model from scratch.
2. Model's hyperparameters were found using the Optuna library using the Tree-structured Parzen Estimator Algorithm in 30 trials.
3. Model was trained/validated/tested using the best parameters and 5-fold CV. Process was repeated for 10 times and the results were averaged and reported at the end.

### Note
1. Ensure that this notebook is in the same working directory as data folder, config.py, utils.py, engine.py and model.py
2. To load models, please download the saved models provided in google drive link from README.md
3. Comment away the training function to load saved models and reproduce results

In [1]:
from utils import seed_everything, LoadHOBDataset
from config import SEED_NO, NUM_FEATURES, NUM_GRAPHS_PER_BATCH, NUM_TARGET, EDGE_DIM, DEVICE, PATIENCE, EPOCHS, N_SPLITS, params_parallel_gnn
from engine import EngineHOB
from model import ParallelGNN

import torch
import numpy as np
import optuna
from sklearn.model_selection import KFold
from torch_geometric.loader import DataLoader
import os 

## 2. Tuning the Model (Suggest to skip and use the already found hyperparameters saved in config.py)

1. Model was tuned using Optuna library following the Tree-structured Parzen Estimator Algorithm for 30 trials. 
2. run_tuning function created to facilitate the train and validation step using an early stopping mechanism 
3. objective function created to facilitate the tuning process

In [5]:
def run_tuning(train_loader, valid_loader, params):
    model = ParallelGNN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_gin_layers=params['num_gin_layers'], num_graph_trans_layers=params['num_graph_trans_layers'], 
                            hidden_size=params['hidden_size'], n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0 

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(f'Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss 
            early_stopping_counter=0
        else:
            early_stopping_counter +=1

        if early_stopping_counter > early_stopping_iter:
            print('Early stopping...')
            break
        print(f'Early stop counter: {early_stopping_counter}')
    
    return best_loss

In [6]:
from sklearn.model_selection import KFold

def objective(trial):
    params = {
        'num_gin_layers' : trial.suggest_int('num_gin_layers', 1,3),
        'num_graph_trans_layers' : trial.suggest_int('num_graph_trans_layers', 1,3),
        'hidden_size' : trial.suggest_int('hidden_size', 64, 512),
        'n_heads' : trial.suggest_int('n_heads', 1, 5),
        'dropout': trial.suggest_float('dropout', 0.1,0.4),
        'learning_rate' : trial.suggest_float('learning_rate', 1e-3, 9e-3, log=True)
    }
    #load dataset 
    dataset_for_cv = LoadHOBDataset(root='./data/graph_data/data_oral_avail_train/', raw_filename='data_oral_avail_train_50.csv')
    kf = KFold(n_splits=N_SPLITS)
    fold_loss = 0

    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        print(f'Fold {fold_no}')
        train_dataset= []
        valid_dataset = []
        for t_idx in train_idx:
            train_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt'))
        for v_idx in valid_idx:
            valid_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt'))
        seed_everything(SEED_NO)
        train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)
        loss = run_tuning(train_loader, valid_loader, params)
        fold_loss+=loss
    return fold_loss/5

if __name__ == '__main__':
    study = optuna.create_study(direction = 'minimize')
    study.optimize(objective, n_trials=30)
    print(f'best trial:')
    trial_ = study.best_trial
    print(trial_.values)
    best_parameters= trial_.params
    print(f'Best parameters: {trial_.params}')

[I 2023-03-08 18:18:09,338] A new study created in memory with name: no-name-7664fac9-a620-45d5-b1e9-d0cc90e84306


Fold 0
Epoch: 1/300, train loss : 2.4939538538455963, validation loss : 0.7173627018928528
Early stop counter: 0
Epoch: 2/300, train loss : 0.7330368608236313, validation loss : 0.643173336982727
Early stop counter: 0
Epoch: 3/300, train loss : 0.7579510807991028, validation loss : 0.7866787314414978
Early stop counter: 1
Epoch: 4/300, train loss : 0.789401575922966, validation loss : 0.7663371562957764
Early stop counter: 2
Epoch: 5/300, train loss : 0.7215721011161804, validation loss : 0.7744593620300293
Early stop counter: 3
Epoch: 6/300, train loss : 0.7517992556095123, validation loss : 0.6887840628623962
Early stop counter: 4
Epoch: 7/300, train loss : 0.6680254936218262, validation loss : 0.6480331420898438
Early stop counter: 5
Epoch: 8/300, train loss : 0.6980799287557602, validation loss : 0.6425188183784485
Early stop counter: 0
Epoch: 9/300, train loss : 0.6851582080125809, validation loss : 0.666217029094696
Early stop counter: 1
Epoch: 10/300, train loss : 0.677630230784

[I 2023-03-08 18:18:38,130] Trial 0 finished with value: 0.639851176738739 and parameters: {'num_gin_layers': 3, 'num_graph_trans_layers': 3, 'hidden_size': 262, 'n_heads': 2, 'dropout': 0.1643323224060143, 'learning_rate': 0.003910770545963381}. Best is trial 0 with value: 0.639851176738739.


Epoch: 17/300, train loss : 0.6131858229637146, validation loss : 0.8122575283050537
Early stopping...
Fold 0
Epoch: 1/300, train loss : 14.046315759420395, validation loss : 2.5939178466796875
Early stop counter: 0
Epoch: 2/300, train loss : 2.439465045928955, validation loss : 0.7556847333908081
Early stop counter: 0
Epoch: 3/300, train loss : 0.863432914018631, validation loss : 0.8677375912666321
Early stop counter: 1
Epoch: 4/300, train loss : 0.7609700560569763, validation loss : 0.6680319905281067
Early stop counter: 0
Epoch: 5/300, train loss : 0.6839437335729599, validation loss : 0.755303144454956
Early stop counter: 1
Epoch: 6/300, train loss : 0.7025461494922638, validation loss : 0.6212304830551147
Early stop counter: 0
Epoch: 7/300, train loss : 0.6692395359277725, validation loss : 0.7014698386192322
Early stop counter: 1
Epoch: 8/300, train loss : 0.678019106388092, validation loss : 0.626433789730072
Early stop counter: 2
Epoch: 9/300, train loss : 0.6762939244508743, 

[I 2023-03-08 18:19:33,793] Trial 1 finished with value: 0.6356054782867432 and parameters: {'num_gin_layers': 2, 'num_graph_trans_layers': 2, 'hidden_size': 480, 'n_heads': 3, 'dropout': 0.3779991183223783, 'learning_rate': 0.0037923301889909054}. Best is trial 1 with value: 0.6356054782867432.


...

Epoch: 13/300, train loss : 0.6362909078598022, validation loss : 0.8439246416091919
Early stop counter: 10
Epoch: 14/300, train loss : 0.6352681815624237, validation loss : 0.8290923833847046
Early stopping...
Fold 0
Epoch: 1/300, train loss : 0.9711443185806274, validation loss : 0.7117790579795837
Early stop counter: 0
Epoch: 2/300, train loss : 0.6864632219076157, validation loss : 0.6813191175460815
Early stop counter: 0
Epoch: 3/300, train loss : 0.6910950243473053, validation loss : 0.6658748984336853
Early stop counter: 0
Epoch: 4/300, train loss : 0.6792733669281006, validation loss : 0.7293267846107483
Early stop counter: 1
Epoch: 5/300, train loss : 0.6814297735691071, validation loss : 0.6351163387298584
Early stop counter: 0
Epoch: 6/300, train loss : 0.6789785325527191, validation loss : 0.6725941896438599
Early stop counter: 1
Epoch: 7/300, train loss : 0.6716247946023941, validation loss : 0.6369011998176575
Early stop counter: 2
Epoch: 8/300, train loss : 0.66643063724

[I 2023-03-08 18:33:13,054] Trial 28 finished with value: 0.6292212843894959 and parameters: {'num_gin_layers': 3, 'num_graph_trans_layers': 3, 'hidden_size': 210, 'n_heads': 1, 'dropout': 0.33163199030560864, 'learning_rate': 0.0019514110839291537}. Best is trial 21 with value: 0.624030876159668.


Epoch: 15/300, train loss : 0.6429032236337662, validation loss : 0.8135614991188049
Early stopping...
Fold 0
Epoch: 1/300, train loss : 0.868861049413681, validation loss : 0.7544718980789185
Early stop counter: 0
Epoch: 2/300, train loss : 0.6900448948144913, validation loss : 0.6496054530143738
Early stop counter: 0
Epoch: 3/300, train loss : 0.6739673167467117, validation loss : 0.6580160856246948
Early stop counter: 1
Epoch: 4/300, train loss : 0.6796716153621674, validation loss : 0.6836300492286682
Early stop counter: 2
Epoch: 5/300, train loss : 0.6751524657011032, validation loss : 0.6502513289451599
Early stop counter: 3
Epoch: 6/300, train loss : 0.6754467040300369, validation loss : 0.6243088245391846
Early stop counter: 0
Epoch: 7/300, train loss : 0.6838980615139008, validation loss : 0.6693140268325806
Early stop counter: 1
Epoch: 8/300, train loss : 0.6720767468214035, validation loss : 0.6575260758399963
Early stop counter: 2
Epoch: 9/300, train loss : 0.66969393193721

[I 2023-03-08 18:33:51,325] Trial 29 finished with value: 0.6365422248840332 and parameters: {'num_gin_layers': 3, 'num_graph_trans_layers': 3, 'hidden_size': 279, 'n_heads': 2, 'dropout': 0.3668419607678221, 'learning_rate': 0.0013495001997228366}. Best is trial 21 with value: 0.624030876159668.


Epoch: 16/300, train loss : 0.6573258489370346, validation loss : 0.700151264667511
Early stopping...
best trial:
[0.624030876159668]
Best parameters: {'num_gin_layers': 3, 'num_graph_trans_layers': 2, 'hidden_size': 248, 'n_heads': 2, 'dropout': 0.343018493669972, 'learning_rate': 0.002091824465119609}


## 3. Train/validate/test model

1. After tuning, the best parameters saved to config.py as params_parallel_gnn
2. Model was trained/validated/tested using the best parameters and 5-fold CV. Process was repeated 5 times and the final results were averaged.
3. Ensure that data folder, from_scratch_trained_models folder and this notebook are in the same working directory
4. run_training to repeat the training process, please create a new folder to retrain, save and load saved model
5. Else, comment the run_training function and do testing to get the results which were obtained in the paper. 

In [3]:
def run_training(train_loader, valid_loader, params, trained_model_path):
    model = ParallelGNN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_gin_layers=params['num_gin_layers'], num_graph_trans_layers=params['num_graph_trans_layers'], hidden_size=params['hidden_size'], 
                            n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB(model, optimizer, device=DEVICE)

    best_loss = np.inf
    early_stopping_iter = PATIENCE
    early_stopping_counter = 0 

    for epoch in range(EPOCHS):
        train_loss = eng.train(train_loader)
        valid_loss = eng.validate(valid_loader)
        print(f'Epoch: {epoch+1}/{EPOCHS}, train loss : {train_loss}, validation loss : {valid_loss}')
        if valid_loss < best_loss:
            best_loss = valid_loss 
            early_stopping_counter=0 #reset counter
            print('Saving model...')
            torch.save(model.state_dict(), trained_model_path)
        else:
            early_stopping_counter +=1

        if early_stopping_counter > early_stopping_iter:
            print('Early stopping...')
            break
        print(f'Early stop counter: {early_stopping_counter}')
    
    return best_loss

def run_testing(test_loader, params, trained_model_path):
    model = ParallelGNN(num_features=NUM_FEATURES, num_targets=NUM_TARGET, num_gin_layers=params['num_gin_layers'], num_graph_trans_layers=params['num_graph_trans_layers'], hidden_size=params['hidden_size'], 
                        n_heads=params['n_heads'], dropout=params['dropout'], edge_dim=EDGE_DIM)
    model.load_state_dict(torch.load(trained_model_path))
    model.to(DEVICE)
    optimizer=torch.optim.Adam(model.parameters(),lr = params['learning_rate'])
    eng = EngineHOB(model, optimizer, device=DEVICE)

    print('Begin testing...')
    bce, acc, f1, roc_auc = eng.test(test_loader)
    print('Test completed!')
    print(f'bce:{bce}, acc:{acc}, f1: {f1}, roc_auc: {roc_auc}')
    return bce, acc, f1, roc_auc

In [4]:
params = params_parallel_gnn
n_repetitions = 5
train_data_root_path = './data/graph_data/data_oral_avail_train/'
train_data_raw_filename = 'data_oral_avail_train_50.csv'
test_data_root_path = './data/graph_data/data_oral_avail_test/'
test_data_raw_filename = 'data_oral_avail_test_1_50.csv'
path_to_save_trained_model = './from_scratch_trained_models/parallel/'

bce_list = []
acc_list = []
f1_list = []
roc_auc_list = []

#load dataset 
dataset_for_cv = LoadHOBDataset(root=train_data_root_path, raw_filename=train_data_raw_filename)
test_dataset = LoadHOBDataset(root=test_data_root_path, raw_filename=test_data_raw_filename)
kf = KFold(n_splits=N_SPLITS)

for repeat in range(n_repetitions): 
    for fold_no, (train_idx, valid_idx) in enumerate(kf.split(dataset_for_cv)):
        seed_everything(SEED_NO)
        train_dataset= []
        valid_dataset = []
        for t_idx in train_idx:
            train_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{t_idx}.pt'))
        for v_idx in valid_idx:
            valid_dataset.append(torch.load(f'./data/graph_data/data_oral_avail_train/processed/molecule_{v_idx}.pt'))

        train_loader = DataLoader(train_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=True)
        valid_loader = DataLoader(valid_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)
        test_loader = DataLoader(test_dataset, batch_size=NUM_GRAPHS_PER_BATCH, shuffle=False)

        run_training(train_loader, valid_loader, params, os.path.join(path_to_save_trained_model, f'parallel_repeat_{repeat}_fold_{fold_no}.pt'))
        bce, acc, f1, roc_auc = run_testing(test_loader, params,  os.path.join(path_to_save_trained_model, f'parallel_repeat_{repeat}_fold_{fold_no}.pt'))
        bce_list.append(bce)
        acc_list.append(acc)
        f1_list.append(f1)
        roc_auc_list.append(roc_auc)

bce_arr = np.array(bce_list)
mean_bce = np.mean(bce_arr)
sd_bce = np.std(bce_arr)
print(f'bce:{mean_bce:.3f}±{sd_bce:.3f}')

acc_arr = np.array(acc_list)
acc_mean= np.mean(acc_arr)
acc_sd = np.std(acc_arr)
print(f'acc:{acc_mean:.3f}±{acc_sd:.3f}')

f1_arr = np.array(f1_list)
f1_mean= np.mean(f1_arr)
f1_sd = np.std(f1_arr)
print(f'f1: {f1_mean:.3f}±{f1_sd:.3f}')

roc_auc_arr = np.array(roc_auc_list)
roc_auc_mean= np.mean(roc_auc_arr)
roc_auc_sd = np.std(roc_auc_arr)
print(f'roc_auc: {roc_auc_mean:.3f}±{roc_auc_sd:.3f}')

Epoch: 1/300, train loss : 1.0919382125139236, validation loss : 0.7346024513244629
Saving model...
Early stop counter: 0
Epoch: 2/300, train loss : 0.6976696997880936, validation loss : 0.6866105198860168
Saving model...
Early stop counter: 0
Epoch: 3/300, train loss : 0.6784620434045792, validation loss : 0.6410688757896423
Saving model...
Early stop counter: 0
Epoch: 4/300, train loss : 0.6682775914669037, validation loss : 0.6641042828559875
Early stop counter: 1
Epoch: 5/300, train loss : 0.6798685938119888, validation loss : 0.646278440952301
Early stop counter: 2
Epoch: 6/300, train loss : 0.6772485971450806, validation loss : 0.6281134486198425
Saving model...
Early stop counter: 0
Epoch: 7/300, train loss : 0.6712305098772049, validation loss : 0.664862334728241
Early stop counter: 1
Epoch: 8/300, train loss : 0.6656370908021927, validation loss : 0.6376354694366455
Early stop counter: 2
Epoch: 9/300, train loss : 0.6571381986141205, validation loss : 0.6277853846549988
Saving